# 📈 NVDA Signals and Forecast

This notebook analyzes NVDA stock historical signals, calculates profitability, and generates a 30-business-day forecast using ARIMA.

---

**Contents**:
- Load and prepare signal data
- Plot signals (green = buy)
- Analyze returns when holding for 5 days
- Forecast next 30 business days using ARIMA


In [ ]:
import pandas as pd

# Load signal data from GitHub (you can also upload manually if local)
url = "https://raw.githubusercontent.com/GiedriusDapsys/nvda-signals-project/main/NVDA_kainos_su_signalais.csv"
df = pd.read_csv(url, parse_dates=["Date"])
df = df.sort_values("Date")
df.head()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
plt.plot(df["Date"], df["Close"], label="NVDA Close Price", color="gray")
plt.scatter(df[df["Signal"] == 1]["Date"], df[df["Signal"] == 1]["Close"], color="green", label="Buy Signal", marker="^", s=80)
plt.title("NVDA Price with Buy Signals")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
signal_days = df[df["Signal"] == 1].copy()
holding_days = 5

signal_days["Exit_Price"] = signal_days["Date"].apply(
    lambda d: df[df["Date"] > d].head(holding_days).Close.values[-1] if len(df[df["Date"] > d].head(holding_days)) == holding_days else None
)

signal_days.dropna(subset=["Exit_Price"], inplace=True)
signal_days["Profit_pct"] = 100 * (signal_days["Exit_Price"] - signal_days["Close"]) / signal_days["Close"]
signal_days[["Date", "Close", "Exit_Price", "Profit_pct"]]


In [ ]:
mean_return = signal_days["Profit_pct"].mean()
total_signals = len(signal_days)

print(f"Total buy signals with full {holding_days}-day holding: {total_signals}")
print(f"Average return per signal: {mean_return:.2f}%")


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

df.set_index("Date", inplace=True)
model = ARIMA(df["Close"], order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)
forecast.plot(figsize=(12,5), title="30 Business Day Forecast")
